In [1]:
pip install flask flask-mysql flask-wtf joblib tensorflow scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install flask mysql-connector-python joblib tensorflow numpy flask-cors

Note: you may need to restart the kernel to use updated packages.


In [7]:
from flask import Flask, render_template, request, redirect, url_for, jsonify
import mysql.connector
import joblib
import tensorflow as tf
import numpy as np
from flask_cors import CORS
import os
import logging

app = Flask(__name__)
CORS(app)

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# MySQL Configuration (use environment variables for sensitive info)
db_config = {
    'host': 'localhost', 
    'user': 'root',  
    'password': 'teju@10', 
    'database': 'agri'
}

# Load Models
crop_model = joblib.load('crop_recommendation_model.pkl')
fertilizer_model = joblib.load('fertilizer_recommendation_model.pkl')
plant_disease_model = tf.keras.models.load_model('plant_disease_model.keras')

# Disease classes for interpretation
DISEASE_CLASSES = {
    0: 'Healthy',
    1: 'Powdery Mildew',
    2: 'Leaf Blight',
    # Add other classes as necessary
}

# Routes

@app.route('/')
def home():
    logger.info("Home page accessed")
    return render_template('home.html')

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        name = request.form['name']
        email = request.form['email']
        phone = request.form['phone']
        connection = mysql.connector.connect(**db_config)
        cursor = connection.cursor()
        try:
            cursor.execute("INSERT INTO farmers (name, email, phone) VALUES (%s, %s, %s)", (name, email, phone))
            connection.commit()
            logger.info(f"New farmer registered: {name}")
        except mysql.connector.Error as err:
            logger.error(f"Error registering farmer: {err}")
            return f"Error: {err}"
        finally:
            cursor.close()
            connection.close()
        return redirect(url_for('home'))
    return render_template('registration.html')

@app.route('/recommend-crop', methods=['GET', 'POST'])
def crop_recommendation():
    if request.method == 'POST':
        data = [
            float(request.form['N']),
            float(request.form['P']),
            float(request.form['K']),
            float(request.form['temperature']),
            float(request.form['humidity']),
            float(request.form['ph']),
            float(request.form['rainfall'])
        ]
        prediction = crop_model.predict([data])
        logger.info(f"Crop recommendation data: {data} | Prediction: {prediction[0]}")
        return jsonify({'Recommended Crop': prediction[0]})
    return render_template('crop_recommendation.html')

@app.route('/recommend-fertilizer', methods=['GET', 'POST'])
def fertilizer_recommendation():
    if request.method == 'POST':
        data = [
            float(request.form['temperature']),
            float(request.form['humidity']),
            float(request.form['moisture']),
            int(request.form['soil_type']),
            int(request.form['crop_type']),
            float(request.form['nitrogen']),
            float(request.form['potassium']),
            float(request.form['phosphorous'])
        ]
        prediction = fertilizer_model.predict([data])
        logger.info(f"Fertilizer recommendation data: {data} | Prediction: {prediction[0]}")
        return jsonify({'Recommended Fertilizer': prediction[0]})
    return render_template('fertilizer_recommendation.html')

@app.route('/detect-disease', methods=['GET', 'POST'])
def disease_detection():
    if request.method == 'POST':
        image_file = request.files['image']
        if image_file:
            image = tf.keras.preprocessing.image.load_img(image_file, target_size=(128, 128))
            image = tf.keras.preprocessing.image.img_to_array(image)
            image = np.expand_dims(image, axis=0) / 255.0
            prediction = plant_disease_model.predict(image)
            predicted_class = np.argmax(prediction, axis=1)[0]
            predicted_label = DISEASE_CLASSES.get(predicted_class, "Unknown Disease")
            logger.info(f"Plant disease detection: {predicted_label}")
            return jsonify({'Predicted Disease': predicted_label})
    return render_template('disease_detection.html')

# Error Handlers

@app.errorhandler(404)
def not_found(error):
    return render_template('404.html'), 404

@app.errorhandler(500)
def internal_error(error):
    return render_template('500.html'), 500

# Test Routes for Debugging

@app.route('/test-crop', methods=['GET'])
def test_crop():
    return jsonify({"message": "Crop recommendation endpoint is working!"})

@app.route('/test-fertilizer', methods=['GET'])
def test_fertilizer():
    return jsonify({"message": "Fertilizer recommendation endpoint is working!"})

@app.route('/test-disease', methods=['GET'])
def test_disease():
    return jsonify({"message": "Disease detection endpoint is working!"})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


INFO:werkzeug: * Restarting with watchdog (windowsapi)


SystemExit: 1